In [14]:
import numpy as np
import pandas as pd
from scipy.stats import norm
N = norm.cdf

In [15]:
options_data1 =pd.read_csv('optiondata_31AUG.csv')
options_data2 =pd.read_csv('optiondata_07SEP.csv')

In [16]:
S = 19265.8      #Current Nifty 50 Index value (as of 27th August)
T1 = 4/252       #Time to expire for the case of options expiring on 31 August 
T2 = 9/252       #Time to expire for the case of options expiring on 07 September
r = 0.1          #Risk-free rate

In [17]:
#Black Scholes model calculations for Call options
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

#Black Scholes model calculations for Put options
def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * N(-d2) - S * N(-d1)

In [18]:
#Predicting the Call option values for 31 August expiry options
options_data1['Predicted CALL'] = options_data1.apply(lambda row: black_scholes_call(S, row['STRIKE'], T1, r, row['IVcall']/100), axis=1)

#Predicting the Put option values for 31 August expiry options
options_data1['Predicted PUT'] = options_data1.apply(lambda row: black_scholes_put(S, row['STRIKE'], T1, r, row['IVput']/100), axis=1)

In [21]:
#Predicting the Call option values for 07 September expiry options
options_data2['Predicted CALL'] = options_data2.apply(lambda row: black_scholes_call(S, row['STRIKE'], T2, r, row['IVcall']/100), axis=1)

#Predicting the Put option values for 07 September expiry options
options_data2['Predicted PUT'] = options_data2.apply(lambda row: black_scholes_put(S, row['STRIKE'], T2, r, row['IVput']/100), axis=1)

In [23]:
#print(options_data1)
#print(options_data2)

In [8]:
#Calculating Mean Absolute Percentage Error of 31 August Call options
options_data1['MAPE CALL'] = abs((options_data1['CALL'] - options_data1['Predicted CALL'])/options_data1['CALL']) * 100
mapecall1 = options_data1['MAPE CALL'].mean()

#Calculating Mean Absolute Percentage Error of 31 August Put options
options_data1['MAPE PUT'] = abs((options_data1['PUT'] - options_data1['Predicted PUT'])/options_data1['PUT']) * 100
mapeput1 = options_data1['MAPE PUT'].mean()

#Calculating Total Mean Absolute Percentage Error of 31 August options
mape1 = (mapecall1 + mapeput1)/2

In [9]:
#Calculating Mean Absolute Percentage Error of 07 September Call options
options_data2['MAPE CALL'] = abs((options_data2['CALL'] - options_data2['Predicted CALL'])/options_data2['CALL']) * 100
mapecall2 = options_data2['MAPE CALL'].mean()

#Calculating Mean Absolute Percentage Error of 07 September Put options
options_data2['MAPE PUT'] = abs((options_data2['PUT'] - options_data2['Predicted PUT'])/options_data2['PUT']) * 100
mapeput2 = options_data2['MAPE PUT'].mean()

#Calculating Total Mean Absolute Percentage Error of 07 September options
mape2 = (mapecall2 + mapeput2)/2

In [24]:
#Taking weighted mean of total error of both the datasets
final_p_error = (options_data1['STRIKE'].count()*mape1 + options_data2['STRIKE'].count()*mape2)/(options_data1['STRIKE'].count() + options_data2['STRIKE'].count())

#Final Accuracy of the implemented Black Scholes Model
final_p_accuracy = 100 - final_p_error
print('Final Accuracy of the Black Scholes Model: {:.2f}%'.format(final_p_accuracy))

Final Accuracy of the Black Scholes Model: 97.24%


THANK YOU!!!